In [16]:
import panel as pn
import pandas as pd
import que
pn.extension(comms='vscode')
import matplotlib.pyplot as plt
import datetime
import yahoo_fin.stock_info as yf
import warnings
from datetime import date
from dateutil.relativedelta import relativedelta
from finquant.portfolio import build_portfolio
from finquant.moving_average import ema
from finquant.moving_average import sma

In [17]:

warnings.filterwarnings('ignore')

In [18]:
def get_company_financials(ticker_list):
    all_financials = {}
    if not ticker_list:
         return "Please provide atleast one ticker"
    else: 
        # avoid looping through dataframes due to loc index issues
        ticker_list = list(ticker_list)
        for tick in ticker_list:
            ticker_report ={}
            dict_financials  = yf.get_financials(tick,yearly =True, quarterly = False)
            ticker_report["income_statement"]= dict_financials['yearly_income_statement'].fillna(0)
            ticker_report["balance_sheet"] = dict_financials['yearly_balance_sheet'].fillna(0)
            ticker_report["yearly_cash_flow"] = dict_financials['yearly_cash_flow'].fillna(0)
            all_financials[tick] = ticker_report
    return all_financials

In [19]:
def get_company_performance(ticker_list):

    all_performance = {}
    if not ticker_list:
         return "Please provide atleast one ticker"
    else: 
        ticker_list = list(ticker_list)
        for tick in ticker_list:
            performance = {}

            try:
                dividend = yf.get_dividends(tick)
                performance["dividend"] = dividend
                earnings = yf.get_earnings_history(tick)
                performance["earnings"] = pd.DataFrame(earnings)
                all_performance[tick] = performance
                # catch exceptions raised on no data available
            except Exception as e : 
                ticker_list.remove(tick)
                #print(f"No dividend for {tick}") 
                
    return all_performance  


In [20]:
def get_financial_stats(ticker_list):
    all_fin_stats = {}
    if not ticker_list:
         return "Please provide atleast one ticker"
    else:
        ticker_list = list(ticker_list) 
        for tick in ticker_list:
            try:
                fin_stats = yf.get_stats(tick)
                all_fin_stats[tick] = fin_stats
            except Exception as e:
                print(e)
                #ticker_list.remove(tick)
    return all_fin_stats

In [21]:
def persona_return_risk_portfolio(persona_number,score_sort,df_div_payout_ordered,df_Beta):
    # 10 year US give bond = ^TNX
    ten_year_bond_symbol = "^TNX"
    # top 2 crypto 
    bitcoin = "BTC-USD"
    etherum = "ETH-USD"
    p_instruments =[]
    weight_instruments ={}
    five_stocks =score_sort[0:5] 
    if persona_number == 1:
        
        
        weight_instruments[ten_year_bond_symbol] = 0.8
        for st in five_stocks:
            weight= (0.2 /5)
            weight_instruments[st[0]] = weight
        p_instruments.append(weight_instruments)
        return p_instruments
    elif persona_number == 2:
        
        
        five_stocks =score_sort[0:5] 
        weight_instruments[ten_year_bond_symbol] = 0.65
        for st in five_stocks:
            weight= (0.35 /5)
            weight_instruments[st[0]] = weight

        p_instruments.append(weight_instruments)
        return p_instruments
    elif persona_number == 3:
        
        four_stocks =score_sort[0:4] 
        weight_instruments[ten_year_bond_symbol] = 0.4
        # 1 share at 20% 
        best_div_payout = df_div_payout_ordered.head(1).loc[0:1,"Ticker"].values[0]
        weight_instruments[best_div_payout] = 0.2

        for st in four_stocks:
            weight= (0.4 /5)
            weight_instruments[st[0]] = weight

        p_instruments.append(weight_instruments)
        return p_instruments
    elif persona_number == 4:
        
        one = score_sort[0:1][0][0] 
        weight_instruments[one] = 0.1 
        weight_instruments[ten_year_bond_symbol] = 0.05
        # 1 share at 20% 
        best_div_payout = df_div_payout_ordered.head(1).loc[0:1,"Ticker"].values[0]
        weight_instruments[best_div_payout] = 0.2
        weight_instruments[bitcoin] = 0.2
        for st in (df_Beta.sort_values(by="BETA", ascending=False).head(3).index):
            weight= (0.65 / 3)
            weight_instruments[st] = weight

        p_instruments.append(weight_instruments) 
        return p_instruments
    elif persona_number ==5:
        
        weight_instruments[ten_year_bond_symbol] = 0.05
        weight_instruments[bitcoin] = (0.45/2)
        weight_instruments[etherum] = (0.45/2)
        for st in (df_Beta.sort_values(by="BETA", ascending=False).head(3).index):
            weight= (0.5 / 3)
            weight_instruments[st] = weight

        p_instruments.append(weight_instruments)
        return p_instruments
    else: 
        return "Persona identifier should be between one and five"

In [22]:
hypothetical = pd.read_csv("./data/hypo_performance.csv",index_col="Plan")
parameters=['bold_rows', 'index', 'header']
df_pane = pn.widgets.DataFrame(hypothetical,name="hypot")
#df_pane

In [23]:
column = pn.Column('# Risk Tolerance Questions ',sizing_mode="stretch_width",)
i=1
dict_answer = dict()
ques_rad =[]
for q in que.questions:
    #print (q['choices'])
    qid = "q" + str(i)
    header = pn.pane.Alert(f"###{q['question']}")
    column.append(header)
    if q == que.q10:
        column.append(df_pane)
    q_options = []
    choices=[]
    for opt in q['choices']:
        q_options.append(opt[1])
        choices.append(opt)         
   
    radio_group = pn.widgets.RadioButtonGroup( name= qid,
        options= q_options,
        button_type='default',)
    ques_rad.append(radio_group)
    column.append(radio_group)
    dict_answer[qid] = choices
    i+=1
    column.append( pn.layout.Divider())    
    

In [24]:
column

BokehModel(combine_events=True, render_bundle={'docs_json': {'6378036e-bfec-4453-9f25-fce5689b743c': {'defs': …

Column(sizing_mode='stretch_width')
    [0] Markdown(str)
    [1] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [2] RadioButtonGroup(name='q1', options=['More than 60', ...], value='More than 60')
    [3] Divider()
    [4] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [5] RadioButtonGroup(name='q2', options=['Within the next year', ...], value='Within the next year')
    [6] Divider()
    [7] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [8] RadioButtonGroup(name='q3', options=['1 week', '1 month', ...], value='1 week')
    [9] Divider()
    [10] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [11] RadioButtonGroup(name='q4', options=['Less than $50,0...], value='Less than $50,000')
    [12] Divider()
    [13] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [14] RadioButtonGroup(name='q5', options=['Very low', 'Below averag...], value='Very low')
    [15] Divider()
    [16] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [17] RadioButtonGroup(name='q6', options=['Deposit it in bank accou...], value='Deposit it in bank accoun...)
    [18] Divider()
    [19] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [20] RadioButtonGroup(name='q7', options=['Absolute loss', ...], value='Absolute loss')
    [21] Divider()
    [22] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [23] RadioButtonGroup(name='q8', options=['I need to see at least a...], value='I need to see a...)
    [24] Divider()
    [25] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [26] RadioButtonGroup(name='q9', options=['Sell all of your remaini...], value='Sell all of y...)
    [27] Divider()
    [28] Alert(str, css_classes=['alert', 'alert-primary']...], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
    [29] DataFrame(name='hypot', value=     Best Case W...)
    [30] RadioButtonGroup(name='q10', options=['Plan A', 'Plan B', ...], value='Plan A')
    [31] Divider()

In [25]:
q_score =[]
for ans in ques_rad:   
    for opt in dict_answer[ans.name] :
        #print(opt) 
        if ans.value in opt:
            q_score.append(int(opt[0]))   
    
print (q_score)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
total_score = sum(q_score)
risk_profile ="Unknown"
risk_persona= 0
if total_score < 20:
       risk_profile = "Preservation"
       risk_persona= 1
elif total_score >= 20 and total_score < 40:
    risk_profile = "Conservative"
    risk_persona= 2
elif total_score >= 40 and total_score < 60:
    risk_profile = "Balanced"
    risk_persona= 3
elif total_score >= 60 and total_score <= 80:
    risk_profile = "Aggressive"
    risk_persona= 4
else:
    risk_profile = "All Equity"
    risk_persona= 5

In [27]:
risk_profile

'Preservation'

In [28]:
persona_number= risk_persona

In [29]:
nasdaq_ticker = pd.read_csv('./data/nasdaq_screener.csv')
nasdaq_ticker.sort_values(['Market Cap'],ascending=False)
fifty_largest= nasdaq_ticker.nlargest(50,'Market Cap')
twenty_largest= nasdaq_ticker.nlargest(20,'Market Cap')
ticker_list_twenty = twenty_largest['Symbol'].to_list()
ticker_list = tuple(ticker_list_twenty)

In [38]:
all_financials={}
all_performance={}
while  not (bool(all_financials)):
    all_financials = get_company_financials(ticker_list_twenty)

In [47]:
#while not (bool(all_performance)):
all_performance = get_company_performance(ticker_list)

In [49]:
all_financial_stats={}
while not (bool(all_financial_stats)):
    all_financial_stats = get_financial_stats(ticker_list_twenty)

In [ ]:
#### Add beta code here

In [66]:
dict_beta = {'BETA': {'AAPL': '1.22',
  'MSFT': '0.79',
  'AMZN': '1.12',
  'GOOG': '1.00',
  'GOOGL': '1.00',
  'FB': '1.29',
  'TSLA': '2.01',
  'NVDA': '1.38',
  'PYPL': '1.15',
  'INTC': '0.66',
  'ASML': '0.93',
  'CMCSA': 1.05,
  'NFLX': 0.79,
  'ADBE': 0.95,
  'CSCO': 0.93,
  'AVGO': 0.99,
  'PEP': 0.62,
  'TXN': 1.07,
  'PDD': 1.5,
  'TMUS': 0.57}}

In [64]:
df_Beta = pd.DataFrame.from_dict(dict_beta)

In [67]:
df_Beta

,BETA
AAPL,1.22
ADBE,0.95
AMZN,1.12
ASML,0.93
AVGO,0.99
CMCSA,1.05
CSCO,0.93
FB,1.29
GOOG,1.00
GOOGL,1.00


In [71]:
list_dfs_FCFE  = []
list_dfs_ND2EB = []
cf_labels = ['netBorrowings','capitalExpenditures','depreciation']
is_labels = ["netIncome",'ebit']

df_ratios = pd.DataFrame()
for tick in all_financials.keys():
    stock_ratios ={}
    yearly_cash_flow = all_financials[tick]["yearly_cash_flow"]
    income_stmnt = all_financials[tick]["income_statement"]
    if (cf_labels[0] in yearly_cash_flow.index) and (cf_labels[1] in yearly_cash_flow.index):
         capex = yearly_cash_flow.loc[cf_labels[1]]
         net_borrow = yearly_cash_flow.loc[cf_labels[0]]
         depreciation = yearly_cash_flow.loc[cf_labels[2]]
    else: 
        #print(tick)
        del all_financials[tick]    
    if is_labels[0] in income_stmnt.index:
        net_income = income_stmnt.loc["netIncome"]
        ebit = income_stmnt.loc["ebit"]
    else: 
        #print(tick)
        del all_financials[tick]
    FCFE = net_income - capex + net_borrow
    ND_to_EB = net_borrow / (ebit + depreciation)
    #print(f"Ticker {tick} has FCFE of {FCFE}")
    df_FCFE = pd.DataFrame(FCFE).transpose()
    df_ND2EB = pd.DataFrame(ND_to_EB).transpose()
    
    df_ND2EB.columns = df_ND2EB.columns.year
    df_ND2EB['Ticker'] = pd.Series(tick, index=df_ND2EB.index)
    list_dfs_ND2EB.append(df_ND2EB)
    
    df_FCFE.columns = df_FCFE.columns.year
    df_FCFE['Ticker'] = pd.Series(tick, index=df_FCFE.index)
    list_dfs_FCFE.append(df_FCFE)

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [ ]:
#concat and clean
df_all_ND2EB = pd.concat(list_dfs_ND2EB).set_index("Ticker")
df_all_ND2EB = df_all_ND2EB.drop(columns=(min(df_all_ND2EB.columns) + 4),axis=1,errors='ignore')
df_all_FCFE = pd.concat(list_dfs_FCFE).set_index("Ticker")
df_all_FCFE= df_all_FCFE.drop(columns=(min(df_all_FCFE.columns) + 4),axis=1,errors='ignore')
df_all_ND2EB.columns.name= "year"
df_all_FCFE.columns.name= "year"

In [ ]:
all_div_payout= []
all_div_coverage=[]

for tick in all_performance.keys():
    
    divi = all_performance[tick]['dividend']
    ann_total_divi = divi.groupby(divi.index.year).sum()
    ann_total_divi = ann_total_divi.T
    ann_total_divi = ann_total_divi[ann_total_divi.columns.intersection(df_all_FCFE.columns)]
   
    earnings = all_performance[tick]['earnings']
    eps = earnings[["startdatetime","epsactual"]]
    eps.loc[:, ("startdatetime")] = pd.to_datetime(eps.loc[:, ("startdatetime")], errors="coerce",format="%Y-%m-%d")
    eps_annual = eps.groupby(eps.loc[:, ("startdatetime")].dt.year).sum()
    eps_annual = eps_annual.T
    eps_annual = eps_annual[eps_annual.columns.intersection(df_all_FCFE.columns)]
    div_coverage = pd.DataFrame(eps_annual.loc['epsactual'] / ann_total_divi.loc["dividend"]).T
    div_coverage["Ticker"] = tick
    div_coverage =div_coverage.set_index("Ticker")
    all_div_coverage.append(div_coverage.dropna())
   
    div_payout =  pd.DataFrame(ann_total_divi.loc["dividend"] / eps_annual.loc['epsactual']).T
    div_payout["Ticker"] = tick
    div_payout =div_payout.set_index("Ticker")
    all_div_payout.append(div_payout.dropna())

In [ ]:
#concat and clean
df_div_payout = pd.concat(all_div_payout)
df_div_payout.columns.name= "year"
df_div_coverage= pd.concat(all_div_coverage)
df_div_coverage.columns.name= "year"
df_div_payout['mean'] = df_div_payout.mean(axis=1)
df_div_payout_ordered = df_div_payout.sort_values(by=['mean'], ascending= False)
df_div_payout_ordered = df_div_payout_ordered.reset_index()
df_div_coverage['mean'] = df_div_coverage.mean(axis=1)
df_div_coverage_ordered = df_div_coverage.sort_values(by=['mean'], ascending= False)
df_div_coverage_ordered = df_div_coverage_ordered.reset_index()
df_all_FCFE['mean'] = df_all_FCFE.mean(axis=1)

In [ ]:
df_all_ND2EB['mean'] = df_all_ND2EB.mean(axis=1)
df_with_divi_FCFE = df_all_FCFE[df_all_FCFE.index.isin(df_div_payout.index)].sort_values(by=['mean'], ascending= False)
# df_with_divi_FCFE["Rank"]= lambda t: .strftime('%d-%b-%Y'), inplace=True)
df_with_divi_FCFE = df_with_divi_FCFE.reset_index()

In [ ]:
df_with_divi_ND2EB = df_all_ND2EB[df_all_ND2EB.index.isin(df_div_payout.index)].sort_values(by=['mean'], ascending= False)
df_with_divi_ND2EB = df_with_divi_ND2EB.reset_index()

In [ ]:
ranked_dfs = []
df_with_divi_ND2EB.name="ND2EB"
df_with_divi_FCFE.name="FCFE"
df_div_coverage_ordered.name="div_coverage"
df_div_payout_ordered.name="div_payout"
ranked_dfs.append(df_with_divi_ND2EB)
ranked_dfs.append(df_with_divi_FCFE)
ranked_dfs.append(df_div_coverage_ordered)
ranked_dfs.append(df_div_payout_ordered)
df_Beta = df_Beta.astype('float')
df_Beta.sort_values(by="BETA",ascending=False).head(3).index

In [ ]:
available_stocks = df_div_payout.index.values
score_each={}
score_each_debug={}
for df in ranked_dfs:
    for tick in available_stocks:
       if tick not in score_each:
            score_each[tick] = 0             
       score_each[tick] = score_each[tick] + df[df["Ticker"] == tick ].index.values[0]

In [ ]:
score_sort = sorted(score_each.items(), key =  lambda kv:(kv[1], kv[0]))

In [ ]:
ten_year_bond_symbol = "^TNX"
# top 2 crypto 
bitcoin = "BTC-USD"
etherum = "ETH-USD"

In [ ]:
threemonth_bond = "^IRX"
bond_price= yf.get_quote_data(threemonth_bond)
risk_free_rate = bond_price ["regularMarketPrice"]

In [ ]:
port = persona_return_risk_portfolio(persona_number, score_sort,df_div_payout_ordered,df_Beta)

In [ ]:
#format portfolio in manner expected by libary 
analytics_dict ={ }
i = 0
for instrument in port:
    for inf in instrument:
        dict_stock = { }
        dict_stock["Name"] = inf
        dict_stock["Allocation"] = instrument[inf] * 100
        analytics_dict[i] = dict_stock
        i+=1

In [ ]:
pf_allocation = pd.DataFrame.from_dict(analytics_dict, orient="index")
names = pf_allocation["Name"].values.tolist()
pf = build_portfolio(
    names=names, pf_allocation=pf_allocation, start_date=ten_years_ago, end_date=today,data_api="yfinance"
)

In [ ]:
graphs ={ }
# plotting cumulative returns (price_{t} - price_{t=0}) / price_{t=0}
plt_cumm_returns, ax = plt.subplots()
ax = pf.comp_cumulative_returns().plot(ax=ax).axhline(y=0, color="black", lw=3)
graphs["Cummulative Returns"] = plt_cumm_returns
#plt.show()



# plotting daily log returns
plt_dlog_returns, ax = plt.subplots()
ax= pf.comp_daily_log_returns().plot(ax=ax).axhline(y=0, color="black")
plt.show()
graphs[" Daily Log Returns" ] = plt_dlog_returns

# cumulative log returns
plt_clog_returns, ax = plt.subplots()
ax = pf.comp_daily_log_returns().cumsum().plot(ax=ax).axhline(y=0, color="black")
plt.show()
graphs[" Cumulative Log Returns" ] = plt_clog_returns

# exponential moving average
plt_ema_returns, ax = plt.subplots()
#ax = pf.data.plot( )
# computing exponential moving average and plotting it
ax= ema(pf.data).plot(ax=ax,grid=True)
plt.show()
graphs[" Exponential Moving Average" ] = plt_ema_returns


# simple moving average
plt_sma_returns_50, ax = plt.subplots()
#ax = pf.data.plot(grid=True)
# computing simple moving average over a span of 50 (trading) days
# and plotting it
ax = sma(pf.data, span=50).plot(ax=ax,grid=True)
plt.show()
graphs[" Simple Moving Average 50 Days " ] = plt_sma_returns_50

# simple moving average
plt_sma_returns_100, ax = plt.subplots()
#ax = pf.data.plot(grid=True)
# computing simple moving average over a span of 50 (trading) days
# and plotting it
ax = sma(pf.data, span=100).plot(ax=ax,grid=True)
plt.show()
graphs[" Simple Moving Average 100 Days" ] = plt_sma_returns_100

# simple moving average
plt_sma_returns_200, ax = plt.subplots()
#ax = pf.data.plot(grid=True)
# computing simple moving average over a span of 50 (trading) days
# and plotting it
ax = sma(pf.data, span=200).plot(ax=ax,grid=True)
plt.show()
graphs[" Simple Moving Average 200 Days" ] = plt_sma_returns_200

In [ ]:
col_indi_ma = pn.Column()

for stock in pf.stocks.keys():
     title = "Moving Averages for "
     st = pf.get_stock(stock).data.copy(deep=True)
     spans = [10, 50, 100, 150, 200]
     st_ma=  compute_ma(st, ema, spans, plot=False)
     plt_st_ma, ax = plt.subplots()
     ax = st_ma.plot(grid=True,ax=ax)
     title = title + stock
     card = pn.Card(plt_st_ma, title=title)
     col_indi_ma.append(card)

col_indi_ma

In [ ]:
risk_free_rate

In [ ]:
current=  252
pf.freq = current
pf.risk_free_rate = risk_free_rate

# 2.a compute and get new values based on new freq/risk_free_rate
exret = pf.comp_expected_return(freq=pf.freq)
vol = pf.comp_volatility(freq=pf.freq)
sharpe = pf.comp_sharpe()
print(
    "For {} trading days and a risk free rate of {}:".format(pf.freq, pf.risk_free_rate)
)
print("Expected return: {:0.3f}".format(exret))
print("Volatility: {:0.3f}".format(vol))
print("Sharpe Ratio: {:0.3f}".format(sharpe))
#str_exp_ret=  "Expected return: {:0.3f}".format(exret)
pn_er = pn.indicators.Number(name='Expected return', value=round(exret,2), format='{value}%',
colors=[(33, 'green')])
pn_vol = pn.indicators.Number(name='Volatility', value=round(vol,2), format='{value}',
colors=[(33, 'red')])
pn_sha =pn.indicators.Number(name='Sharpe Ratio', value=round(sharpe,2), format='{value}',
colors=[(33, 'orange')])
row_current = pn.Row(pn_er,pn_vol,pn_sha)

In [ ]:
three_years=  252*3
pf.freq = three_years
pf.risk_free_rate = risk_free_rate

# 2.a compute and get new values based on new freq/risk_free_rate
exret = pf.comp_expected_return(freq=pf.freq)
vol = pf.comp_volatility(freq=pf.freq)
sharpe = pf.comp_sharpe()
print(
    "For {} trading days and a risk free rate of {}:".format(pf.freq, pf.risk_free_rate)
)
print("Expected return: {:0.3f}".format(exret))
print("Volatility: {:0.3f}".format(vol))
print("Sharpe Ratio: {:0.3f}".format(sharpe))
#str_exp_ret=  "Expected return: {:0.3f}".format(exret)
pn_er_three = pn.indicators.Number(name='Expected return', value=round(exret,2), format='{value}%',
colors=[(33, 'green')])
pn_vol_three = pn.indicators.Number(name='Volatility', value=round(vol,2), format='{value}',
colors=[(33, 'red')])
pn_sha_three =pn.indicators.Number(name='Sharpe Ratio', value=round(sharpe,2), format='{value}',
colors=[(33, 'orange')])
row_curr= pn.Row(pn_er_three,pn_vol_three,pn_sha_three)

In [ ]:
five_years=  252*5
pf.freq = five_years
pf.risk_free_rate = risk_free_rate

# 2.a compute and get new values based on new freq/risk_free_rate
exret = pf.comp_expected_return(freq=pf.freq)
vol = pf.comp_volatility(freq=pf.freq)
sharpe = pf.comp_sharpe()
print(
    "For {} trading days and a risk free rate of {}:".format(pf.freq, pf.risk_free_rate)
)
print("Expected return: {:0.3f}".format(exret))
print("Volatility: {:0.3f}".format(vol))
print("Sharpe Ratio: {:0.3f}".format(sharpe))
pn_er_five=pn.indicators.Number(name='Expected return', value=round(exret,2), format='{value}%',
colors=[(33, 'green')])
pn_vol_five=pn.indicators.Number(name='Volatility', value=round(vol,2), format='{value}',
colors=[(33, 'red')])
pn_sha_five=pn.indicators.Number(name='Sharpe Ratio', value=round(sharpe,2), format='{value}',
colors=[(33, 'orange')])
row_five_years= pn.Row(pn_er_five,pn_vol_five,pn_sha_five)

In [ ]:
ten_years_days=  252*10
pf.freq = ten_years_days
pf.risk_free_rate = risk_free_rate

# 2.a compute and get new values based on new freq/risk_free_rate
exret = pf.comp_expected_return(freq=pf.freq)
vol = pf.comp_volatility(freq=pf.freq)
sharpe = pf.comp_sharpe()
print(
    "For {} trading days and a risk free rate of {}:".format(pf.freq, pf.risk_free_rate)
)
print("Expected return: {:0.3f}".format(exret))
print("Volatility: {:0.3f}".format(vol))
print("Sharpe Ratio: {:0.3f}".format(sharpe))
pn_er_ten= pn.indicators.Number(name='Expected return', value=round(exret,2), format='{value}%',
colors=[(33, 'green')])
pn_vol_ten=pn.indicators.Number(name='Volatility', value=round(vol,2), format='{value}',
colors=[(33, 'red')])
pn_sha_ten=pn.indicators.Number(name='Sharpe Ratio', value=round(sharpe,2), format='{value}',
colors=[(33, 'orange')])
row_ten_years= pn.Row(pn_er_ten,pn_vol_ten,pn_sha_ten)

In [ ]:
row1= pn.Row()
row2= pn.Row()
column= pn.Column()
col = 0
for graph in graphs.keys():
    #mpl_pane = pn.pane(graphs[graph], dpi=144)
    card = pn.Card(graphs[graph], title=graph)
    if col < 3:
        row1.append(card)
        col += 1
    else:
        row2.append(card)
        col += 1

column.append(row1)
column.append(row2)

In [ ]:
   row_returns = pn.Column(
        pn.Card(row_curr, title="Current Year"),
        pn.Card(row_three_years, title="Three Years"),
        pn.Card(row_five_years, title="Five Years"),
        pn.Card(row_five_years, title="Ten Years")
    )

In [ ]:
tabs = pn.Tabs(
('Returns Information', row_returns),    
('Portfolio Anlaysis', column),
("Individual Stock Analysis",col_indi_ma),
)
tabs